# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Output File (CSV)
output_data_file = "Resources/city_data.csv"
city_df = pd.read_csv(output_data_file, encoding="ISO-8859-1")
city_df

,City,Country,City ID,Latitude,Longitude,temperature,humidity,cloudiness,wind speed
0,Te Anau,NZ,2181625,-45.4167,167.7167,75.51,39,0,5.66
1,Mataura,NZ,6201424,-46.1927,168.8643,73.87,53,16,7.65
2,Nanortalik,GL,3421765,60.1432,-45.2371,28.99,94,100,11.14
3,Kapaa,US,5848280,22.0752,-159.3190,72.82,78,20,18.41
4,Hearst,CA,5973108,49.6835,-83.6665,26.60,93,89,2.30
...,...,...,...,...,...,...,...,...,...
561,Mount Isa,AU,2065594,-20.7333,139.5000,96.80,23,0,13.80
562,Zhob,PK,1162105,31.3411,69.4481,48.61,40,3,0.83
563,Formosa,AR,3433899,-26.1775,-58.1781,80.60,78,0,8.05
564,Tofield,CA,6167254,53.3668,-112.6686,28.99,80,67,21.85


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_df[["Latitude", "Longitude"]].astype(float)

# Convert Humidity to float and store
# HINT: be sure to handle NaN values
city_df = city_df.dropna()
humidity = city_df["humidity"].astype(float)

In [5]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

ValueError: weights must be of the same length as locations or None

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
narrow_df = city_df.loc[(city_df["temperature"] < 90) & (city_df["temperature"] > 75) 
                       & (city_df["wind speed"] < 5) & (city_df["cloudiness"] < 5)]
narrow_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = narrow_df
hotel_df['Hotel Name'] = ""
hotel_df

In [ ]:
#06-Python-APIs/3/Activities/04-Ins_NearestRestr/Solved/NearestRestr.ipynb
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


params = { 
    "radius":5000,
    "type": "hotel",
    "keyword" : "hotel",
    "key": g_key,
}

In [ ]:
for index, row in hotel_df.iterrows():
    
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try: 
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Hotel not found.")
        
    print("------------")

In [ ]:
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [ ]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig